<a href="https://colab.research.google.com/github/JulianBrinez/Segundo-Parcial/blob/main/Segundo%20punto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Julian Briñez Segundo parcial


**PUNTO 2 SEGUNDO PARCIAL**

In [23]:
!pip install mesa



In [37]:
from mesa import Agent, Model
import re
import queue

class OperationAgent(Agent):
    def __init__(self, unique_id, model, operation):
        super().__init__(unique_id, model)
        self.operation = operation

    def operar(self, a, b):
        if self.operation == '+':
            return a + b
        elif self.operation == '-':
            return a - b
        elif self.operation == '*':
            return a * b
        elif self.operation == '/':
            return a / b if b != 0 else "Error: División por cero"

class CommunicationAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.mensajes = queue.Queue()

    def enviar_mensaje(self, mensaje):
        self.mensajes.put(mensaje)

    def recibir_mensaje(self):
        if not self.mensajes.empty():
            return self.mensajes.get()
        return None

class CalculadoraModel(Model):
    def __init__(self):
        self.schedule = None
        self.agentes = {
            '+': OperationAgent(1, self, '+'),
            '-': OperationAgent(2, self, '-'),
            '*': OperationAgent(3, self, '*'),
            '/': OperationAgent(4, self, '/'),
            'comunicacion': CommunicationAgent(5, self)
        }

    def resolver_expresion(self, expresion):
        tokens = re.findall(r'\d+\.\d+|\d+|[+\-*/()]', expresion)
        postfix = self._infix_a_postfix(tokens)
        return self._evaluar_postfix(postfix)

    def _infix_a_postfix(self, tokens):
        precedencia = {'+': 1, '-': 1, '*': 2, '/': 2}
        output = []
        operadores = []

        for token in tokens:
            if re.match(r'\d+\.\d+|\d+', token):
                output.append(float(token) if '.' in token else int(token))
            elif token in precedencia:
                while (operadores and operadores[-1] in precedencia and
                       precedencia[operadores[-1]] >= precedencia[token]):
                    output.append(operadores.pop())
                operadores.append(token)
            elif token == '(':
                operadores.append(token)
            elif token == ')':
                while operadores and operadores[-1] != '(':
                    output.append(operadores.pop())
                operadores.pop()

        while operadores:
            output.append(operadores.pop())

        return output

    def _evaluar_postfix(self, postfix):
        pila = []

        for token in postfix:
            if isinstance(token, (int, float)):
                pila.append(token)
            else:
                b = pila.pop()
                a = pila.pop()
                resultado = self.agentes[token].operar(a, b)
                self.agentes['comunicacion'].enviar_mensaje(f'Resultado de {a} {token} {b} es {resultado}')
                pila.append(resultado)

        return pila[0] if pila else "Error en la expresión"

calculadora = CalculadoraModel()
resultado = calculadora.resolver_expresion("2 + 3 * 4 - 5")
print("Resultado de 2 + 3 * 4 - 5:", resultado)

comunicacion_agent = calculadora.agentes['comunicacion']
while not comunicacion_agent.mensajes.empty():
    print(comunicacion_agent.recibir_mensaje())


Resultado de 2 + 3 * 4 - 5: 9
Resultado de 3 * 4 es 12
Resultado de 2 + 12 es 14
Resultado de 14 - 5 es 9
